In [5]:
import re, os, errno
from tqdm.notebook import tqdm

## Nombres de boletines

### Boletines RAC

El formato del nombre d elos archivos era irregular en el tema de los espacios.
A veces las fechas tenían un espacio extra entre el guión anterior al año y este ("- 2018").

In [2]:
ruta_rel = os.path.join('..', 'Corpus', 'RAC', 'txt', 'originales')
ruta_abs = os.path.abspath(ruta_rel)

# Quitamos los directorios
archivos = [
    f for f in os.listdir(ruta_abs) 
    if os.path.isfile(os.path.join(ruta_abs, f))
]

archivos[:5]

['COMUNICADO_01-RAC-30-03-2018.txt',
 'COMUNICADO_02-RAC-31-03-2018.txt',
 'COMUNICADO_03-RAC-31-03-2018.txt',
 'COMUNICADO_04-RAC-01-04-2018.txt',
 'COMUNICADO_05-RAC-01-04-2018.txt']

In [6]:
for filename in archivos:
    archivo_original=os.path.join(ruta_abs, filename)
#     print(filename)
    remplazo = re.sub(
        'COMUNICADO (?P<digit>[\d]+)', 
        'COMUNICADO_\g<digit>', 
        filename
    )
    remplazo = re.sub(' ', '', remplazo)
    if remplazo != filename:
        print(remplazo)
        archivo_remplazo=os.path.join(ruta_abs, remplazo)
        os.rename(archivo_original, archivo_remplazo)

### Boletines JAMK

El formato de fecha en el nombre de los boletines a veces marcaba 2019 cuando todos son del 2018.

In [4]:
ruta_rel = os.path.join('..', 'Corpus', 'JAMK', 'txt', 'originales')
ruta_abs = os.path.abspath(ruta_rel)

# Quitamos los directorios
archivos = [
    f for f in os.listdir(ruta_abs) 
    if os.path.isfile(os.path.join(ruta_abs, f))
]

archivos[:5]

['010418 JAMK miles siguen inicio campaña.txt',
 '010418 JAMK Yuc candidato cerca jóvenes.txt',
 '010418 JAMK Yuc presenta 7 compr Mex.txt',
 '010518 JAMK Edomex unidad.txt',
 '010618 Eurviel Avila Edomex Ecatepec.txt']

In [4]:
for filename in archivos:
    archivo_original=os.path.join(ruta_abs, filename)
    remplazo = re.sub(
        '^(?P<digit>[\d]{4})19', 
        '\g<digit>18', 
        filename
    )
    if remplazo != filename:
        print(remplazo)
        archivo_remplazo=os.path.join(ruta_abs, remplazo)
        os.rename(archivo_original, archivo_remplazo)

## Contenido boletines

### Eliminar elementos no léxicos

Elimino espacios, saltos de línea, símbolos especiales, etc

In [7]:
def preprocesar(dir_origen, archivos, dir_destino):
    try:        
        os.makedirs(dir_destino) # Creo directorio de destino
    except OSError as e: 
        if e.errno != errno.EEXIST: # Si ya existe no hago nada
            raise # Si ocurrió otro error, levanto excepción
    remplazos = [
        # Elimino bigotes al final de archivos de AMLO
        (r'\u00aa(\u00aa)+', ''),
        # Elimino bigotes al final de archivos de JAMK
        (r'OO(O)+', ''),
        # Elimino bigotes al final de archivos de RAC
        (re.escape('**00**'), ''),
        (r'\*\*(\*)+', ''),
        # Remplaza cadenas encerradas dentro de doble guión bajo,
        # por ejemplo: _Hola_, _cómo estás_
        (r'_(?P<word>[^_]+)_', r'\g<word>'),
        # Remplazo caracter &nbsp (\u00A0) y tabuladores (\t) por espacio
        (r'[\u00A0\t]', ' '),
        # Remplazo repeticiones de más de dos espacios por espacio simple
        (r' ( )+', ' '),
        # Quito los puntos que quedaron huéfanos en una sóla línea
        (r'(\n\.)', ''),
        # Quito espacios al final e inicio de nuevas líneas
        (r'(( +)\n)|(\n( +))', '\n'),
        # Remplazo saltos de líneas múltiples por un identificador especial
        (r'\n(\n)+', '<dnl>'),
        # Cambio saltos de linea simples por espacios
        (r'\n', ' '),
        # Remplazo identificador de saltos de líneas múltiple por linea simple
        (r'<dnl>', '\n'),
        # Elimina links a imágenes tipo [C:\Hola\Mundo.jpg]
        (r'\[(.+)+\]', ''),
        # Elimino tablas al inicio
        (r'^\+.*\+\n', ''),
        # Elimino nuevas lineas y espacios al inicio y fin de archivo
        (r'(^[ \n]+)|([ \n]+$)', ''),
        # Estandarizo el uso de comillas dobles a “Cita chidorina”
        # Caracteres \u201c “  \u201d ”
        (r'(^|[ \n])\u201d([0-9\wáéíóúÁÉÍÓÚüÜÑñ])', r'\1“\2'),
        (r'(^|[ \n])"([0-9\wáéíóúÁÉÍÓÚüÜÑñ])', r'\1“\2'),
        (r'([0-9\wáéíóúÁÉÍÓÚüÜÑñ])\u201c($|[ ,\.\n])', r'\1”\2'),
        (r'([0-9\wáéíóúÁÉÍÓÚüÜÑñ])"($|[ ,\.\n])', r'\1”\2'),
        # Cambio dos guiones por guión por simple
        (r'--+', r'—'),
        # Cambio estructuras -hola mundo- por —Hola mundo— (caracter \u2014)
        (r'(^|[ \n])-([0-9\wáéíóúÁÉÍÓÚüÜÑñ])', r'\1—\2'),
        (r'([0-9\wáéíóúÁÉÍÓÚüÜÑñ])-($|[ ,\.\n])', r'\1—\2'),
        # Elimino caracter de acento agudo
        (r'\u00B4', ''),
        # Cambio comillas simples \u2018 y \u2019 por '
        (r'[\u2018\u2019]', r"'"),
    ]
    
    # Compilo los patrones
    remplazos = [
        (re.compile(patron), remplazo) 
        for patron, remplazo in remplazos
    ]
    
    # Proceso los archivos
    for fname in tqdm(archivos):
        contenido = ""
        
        # Leo archivos de entrada
        fname_origen = os.path.join(dir_origen, fname)
        tqdm.write(f'Procesando: {fname}...')
        with open(fname_origen, 'r', encoding='utf-8') as f:
            contenido = f.read()
        
        # Proceso patrones
        for patron, remplazo in remplazos:
            contenido = patron.sub(remplazo, contenido)
        
        # Escribo los archivos
        fname_destino = os.path.join(dir_destino, fname)
        with open(fname_destino, 'w', encoding='utf-8') as wf:
            wf.write(contenido)

In [8]:
ruta_folder = os.path.join(ruta_abs, 'preprocesados')
for candidato in ['AMLO', 'RAC', 'JAMK']:
    ruta_base = os.path.join('..', 'Corpus', candidato, 'txt')
    ruta_base = os.path.abspath(ruta_base)

    ruta_origen = os.path.join(ruta_base, 'originales')
    ruta_destino = os.path.join(ruta_base, 'preprocesados')
    
    # Solo proceso archivos, no directorios.
    archivos = [
        f for f in os.listdir(ruta_origen) 
        if os.path.isfile(os.path.join(ruta_origen, f))
    ] 
    
    preprocesar(ruta_origen, archivos, ruta_destino)

Procesando: Boletín-campaña-002-1_020418.txt...
Procesando: Boletín-campaña-004_040418.txt...
Procesando: Boletín-campaña-005_040418.txt...
Procesando: Boletín-campaña-006_040418.txt...
Procesando: Boletín-campaña-007_050418.txt...
Procesando: Boletín-campaña-008-1_050418.txt...
Procesando: Boletín-campaña-009-1_060418.txt...
Procesando: Boletín-campaña-010_070418.txt...
Procesando: Boletín-campaña-011-1_080418.txt...
Procesando: Boletín-campaña-013-1_090418.txt...
Procesando: Boletín-campaña-015_100418.txt...
Procesando: Boletín-campaña-016-1_110418.txt...
Procesando: Boletín-campaña-017_120418.txt...
Procesando: Boletín-campaña-019_130418.txt...
Procesando: Boletín-campaña-020_130418.txt...
Procesando: Boletín-campaña-021_140418.txt...
Procesando: Boletín-campaña-022_130418.txt...
Procesando: Boletín-campaña-023_150418.txt...
Procesando: Boletín-campaña-024-2_160418.txt...
Procesando: Boletín-campaña-028_190418.txt...
Procesando: Boletín-campaña-029_200418.txt...
Procesando: Boletín-

Procesando: COMUNICADO_01-RAC-30-03-2018.txt...
Procesando: COMUNICADO_02-RAC-31-03-2018.txt...
Procesando: COMUNICADO_03-RAC-31-03-2018.txt...
Procesando: COMUNICADO_04-RAC-01-04-2018.txt...
Procesando: COMUNICADO_05-RAC-01-04-2018.txt...
Procesando: COMUNICADO_06-RAC-02-04-2018.txt...
Procesando: COMUNICADO_07-RAC-03-04-2018.txt...
Procesando: COMUNICADO_08-RAC-03-04-2018.txt...
Procesando: COMUNICADO_09-RAC-03-04-2018.txt...
Procesando: COMUNICADO_10-RAC-04-04-2018.txt...
Procesando: COMUNICADO_100-RAC-14-06-2018.txt...
Procesando: COMUNICADO_101-FRENTE-15-06-2018.txt...
Procesando: COMUNICADO_102-RAC-15-06-2018.txt...
Procesando: COMUNICADO_103-FRENTE-15-06-2018.txt...
Procesando: COMUNICADO_104-RAC-16-06-2018.txt...
Procesando: COMUNICADO_106-RAC-18-06-2018.txt...
Procesando: COMUNICADO_109-RAC-19-06-2018.txt...
Procesando: COMUNICADO_11-RAC-04-04-2018.txt...
Procesando: COMUNICADO_111-FRENTE-20-06-2018.txt...
Procesando: COMUNICADO_112-RAC-20-06-2018.txt...
Procesando: COMUNICADO

Procesando: 010418 JAMK miles siguen inicio campaña.txt...
Procesando: 010418 JAMK Yuc candidato cerca jóvenes.txt...
Procesando: 010418 JAMK Yuc presenta 7 compr Mex.txt...
Procesando: 010518 JAMK Edomex unidad.txt...
Procesando: 010618 Eurviel Avila Edomex Ecatepec.txt...
Procesando: 010618 JAMK Jalisco compromisos por Jal.txt...
Procesando: 020418 JAMK Sinaloa recibimiento candidato.txt...
Procesando: 020418 JAMK Sonora 7 compromisos por Mex.txt...
Procesando: 020418 JAMK Sonora bienvenida a Meade.txt...
Procesando: 020518 JAMK Pue Plan Nal Transp.txt...
Procesando: 020618 JAMK Colima Sin amnistía.txt...
Procesando: 030418 JAMK Edomex Juana indígenas.txt...
Procesando: 030418 JAMK Sinaloa Avanzar Contigo.txt...
Procesando: 030518 JAMK Mich defensa migrantes.txt...
Procesando: 030618 JAMK Hidalgo jefas de familia.txt...
Procesando: 040418 JAMK Dgo desarrollo Edo.txt...
Procesando: 040418 JAMK Dgo red por la eco familiar.txt...
Procesando: 040418 JAMK Hgo Juana con mujeres trab.txt...